In [1]:
import subprocess
import os

import requests
from rdflib import Graph, URIRef
from rdflib.namespace import Namespace, OWL, RDF, RDFS
from dotenv import load_dotenv

from ir_onto_demos_utilities import (
    show_instances, show_owl_classes, show_owl_obj_props, 
    sparql_update, sparql_select, insert_graph, delete_graph,
    show_graph_portion
)

load_dotenv()

True

In [2]:
JAVA_EXE = os.getenv("JAVA_EXE")
ROBOT_JAR = os.getenv("ROBOT_JAR")
HERMIT_JAR = os.getenv("HERMIT_JAR")

The main goal of this demo is to demonstrate the power of the IR ontology. In a subsequent demo, we will demonstrate how one can leverage the same data graph for a separate purpose by simply applying another domain graph, i.e., another point of view on the same data.

In this demo we focus on:

- Clearly distinguishing the domain, mapping, and data graphs
- Constructing a custom domain graph by extending an existing upper ontology, e.g., the Information Retrieval ontology
- Leveraging SPARQL construct queries to build a mapping graph
- Demonstrating how one can leverage the IR ontology logic:
  - Analysing user queries to find semantically inconsistent ones
  - Spotting some inconsistent documents
  - Running a DL-based facetted search


In [3]:
# define some namespaces
pizza_ns = Namespace("http://www.msesboue.org/o/pizza-data-demo/bisou#")
ir_onto_ns = Namespace("http://www.msesboue.org/o/ir-ontology#")

In [4]:
pizza_bisou_g = Graph()
pizza_bisou_g.bind(prefix="pizza", namespace=pizza_ns)
pizza_bisou_g.parse("./data/pizzas_bisou.ttl")

<Graph identifier=N6b4c35ea6b084b65a978209ccb8d984e (<class 'rdflib.graph.Graph'>)>

In [5]:
ingredients_g = Graph()
ingredients_g.bind(prefix="pizza", namespace=pizza_ns)
ingredients_g.parse("./data/pizzas_bisou_ingredients.ttl")

<Graph identifier=Nfd056ff1fd5647238c744fe2d05874b5 (<class 'rdflib.graph.Graph'>)>

In [6]:
ir_onto_g = Graph()
ir_onto_g.bind(prefix="ir-onto", namespace=ir_onto_ns)
ir_onto_g.parse("../information-retrieval-ontology.ttl")

<Graph identifier=N6b94daec5544471f926b934a225b3def (<class 'rdflib.graph.Graph'>)>

In [7]:
data_graph = pizza_bisou_g + ingredients_g

In [8]:
map_to_ir_onto_g = Graph()
map_to_ir_onto_g.bind(prefix="pizza", namespace=pizza_ns)
map_to_ir_onto_g.bind(prefix="ir-onto", namespace=ir_onto_ns)
# map_to_ir_onto_g.parse("./data/pizzas_bisou_map_to_ir_onto.ttl")

In [9]:
# Make the triples actual named individual and object properties
query = """
    CONSTRUCT {
        ?s   rdf:type    owl:NamedIndividual .
        ?p   rdf:type    owl:ObjectProperty .
        ?o   rdf:type    owl:NamedIndividual .
    } WHERE {
        ?s  ?p  ?o .
    }
"""

for triple in data_graph.query(query):
    map_to_ir_onto_g.add(triple)

In [10]:
# We have 5 types of instances: Pizza, PizzaTopping, PizzaBase, Spiciness, and PizzaKind

# SHOULD BE PART OF THE DOMAIN GRAPH
# THE DOMAIN GRAPH IS A COMBINATION OF THE IR ONTO AND THE PIZZA DOMAIN
map_to_ir_onto_g.add((pizza_ns.Pizza, RDF.type, OWL.Class))
map_to_ir_onto_g.add((pizza_ns.PizzaTopping, RDF.type, OWL.Class))
map_to_ir_onto_g.add((pizza_ns.PizzaBase, RDF.type, OWL.Class))
map_to_ir_onto_g.add((pizza_ns.Spiciness, RDF.type, OWL.Class))
map_to_ir_onto_g.add((pizza_ns.PizzaKind, RDF.type, OWL.Class))

<Graph identifier=N43e96756c6a74589bae2bd1bc6421b3a (<class 'rdflib.graph.Graph'>)>

In [11]:
# Each pizzas in our data graph has a base
# So this construct query will make sure each pizza is defined as an instance of the class Pizza.

q_construct_pizza = """
    CONSTRUCT {
        ?pizza   rdf:type    pizza:Pizza .
    } WHERE {
        ?pizza  pizza:has_base  ?base .
    }
"""
for triple in pizza_bisou_g.query(q_construct_pizza): # This query is made over the pizza graph
    print(triple[0]) # print the pizzas URI
    map_to_ir_onto_g.add(triple)

http://www.msesboue.org/o/pizza-data-demo/bisou#_mortadella
http://www.msesboue.org/o/pizza-data-demo/bisou#_hotStuff40
http://www.msesboue.org/o/pizza-data-demo/bisou#_quatroFromaggi
http://www.msesboue.org/o/pizza-data-demo/bisou#_bambino
http://www.msesboue.org/o/pizza-data-demo/bisou#_tartufo
http://www.msesboue.org/o/pizza-data-demo/bisou#_leonardo
http://www.msesboue.org/o/pizza-data-demo/bisou#_laVieEnRose
http://www.msesboue.org/o/pizza-data-demo/bisou#_godSaveTheKing
http://www.msesboue.org/o/pizza-data-demo/bisou#_margherita20
http://www.msesboue.org/o/pizza-data-demo/bisou#_burraTadah
http://www.msesboue.org/o/pizza-data-demo/bisou#_biancaCastafiore
http://www.msesboue.org/o/pizza-data-demo/bisou#_zucchero


In [12]:
# Let us find the top categories in our taxonomies
# A top category is a node in our data graph which do not have an in coming "has subcategory" link

q_top_categories = """
    SELECT DISTINCT ?top_cate WHERE {
        ?top_cate  pizza:has_subcategory  ?subcate .
        FILTER NOT EXISTS {
            ?other_cate  pizza:has_subcategory  ?top_cate .
        }
    }
"""
for r in ingredients_g.query(q_top_categories): # This query is made over the ingredient graph
    print(r["top_cate"]) # print the pizzas URI

http://www.msesboue.org/o/pizza-data-demo/bisou#_pizzaBase
http://www.msesboue.org/o/pizza-data-demo/bisou#_pizzaKind
http://www.msesboue.org/o/pizza-data-demo/bisou#_pizzaSpiciness
http://www.msesboue.org/o/pizza-data-demo/bisou#_pizzaTopping


In [13]:
# Let us group the categories by their taxonomies
# for each top category we will assert the top category and its children (has subcategory)
# as an instance of the corresponding top category class
# Note that we could have used a punning approach for the top level category but we wish 
# to keep this demo as simple as possible for the moment. 

# Note that we could also use reasoning by simply declaring a domain and a range for each object properties
# But it a way more restrictive approach that we might not want

q_construct_pizza_kinds = """
    CONSTRUCT {
        ?kind   rdf:type    pizza:PizzaKind .
    } WHERE {
        pizza:_pizzaKind  pizza:has_subcategory*  ?kind .
    }
"""
for triple in ingredients_g.query(q_construct_pizza_kinds): # This query is made over the ingredient graph
    # print(triple[0]) # print the pizza kind URIs
    map_to_ir_onto_g.add(triple)

q_construct_pizza_bases = """
    CONSTRUCT {
        ?base   rdf:type    pizza:PizzaBase .
    } WHERE {
        pizza:_pizzaBase  pizza:has_subcategory*  ?base .
    }
"""
for triple in ingredients_g.query(q_construct_pizza_bases): # This query is made over the ingredient graph
    print(triple[0]) # print the pizza base URIs
    map_to_ir_onto_g.add(triple)

q_construct_pizza_spiciness = """
    CONSTRUCT {
        ?kind   rdf:type    pizza:PizzaSpiciness .
    } WHERE {
        pizza:_pizzaSpiciness  pizza:has_subcategory*  ?kind .
    }
"""
for triple in ingredients_g.query(q_construct_pizza_spiciness): # This query is made over the ingredient graph
    # print(triple[0]) # print the pizza spiciness URIs
    map_to_ir_onto_g.add(triple)

q_construct_pizza_toppings = """
    CONSTRUCT {
        ?topping   rdf:type    pizza:PizzaTopping .
    } WHERE {
        pizza:_pizzaToppings  pizza:has_subcategory*  ?topping .
    }
"""
for triple in ingredients_g.query(q_construct_pizza_toppings): # This query is made over the ingredient graph
    # print(triple[0]) # print the pizza spiciness URIs
    map_to_ir_onto_g.add(triple)

http://www.msesboue.org/o/pizza-data-demo/bisou#_pizzaBase
http://www.msesboue.org/o/pizza-data-demo/bisou#_blackTruffleCreamBase
http://www.msesboue.org/o/pizza-data-demo/bisou#_mustardCreamBase
http://www.msesboue.org/o/pizza-data-demo/bisou#_onionSquashCreamBase
http://www.msesboue.org/o/pizza-data-demo/bisou#_tomatoSauceAlaNdujaBase
http://www.msesboue.org/o/pizza-data-demo/bisou#_creamBase
http://www.msesboue.org/o/pizza-data-demo/bisou#_ricottaCreamBase
http://www.msesboue.org/o/pizza-data-demo/bisou#_tomatoBase
http://www.msesboue.org/o/pizza-data-demo/bisou#_oignonCreamBase


In [14]:
# Let us now add the links to the IR ontology
# In our example we categorise pizzas by country, base, kind, topping and spiciness
map_to_ir_onto_g.add((pizza_ns.PizzaBase, RDFS.subClassOf, ir_onto_ns.Category))
map_to_ir_onto_g.add((pizza_ns.PizzaKind, RDFS.subClassOf, ir_onto_ns.Category))
map_to_ir_onto_g.add((pizza_ns.PizzaTopping, RDFS.subClassOf, ir_onto_ns.Category))
map_to_ir_onto_g.add((pizza_ns.Spiciness, RDFS.subClassOf, ir_onto_ns.Category))

# In our example the search engine goal is to find pizzas
map_to_ir_onto_g.add((pizza_ns.Pizza, RDFS.subClassOf, ir_onto_ns.Document))

# The pizza:has_subcategory relation is the same as the IR ontology one
map_to_ir_onto_g.add((pizza_ns.has_subcategory, OWL.equivalentProperty, ir_onto_ns.hasSubcategory))

# In our example when a category is selected we want ot have the subcategories enabled --> SHOULD BE PART OF OUR DOMAIN GRAPH
map_to_ir_onto_g.add((ir_onto_ns.hasSubcategory, RDFS.subPropertyOf, ir_onto_ns.enablesCategory))
map_to_ir_onto_g.add((ir_onto_ns.hasSubcategory, RDF.type, OWL.TransitiveProperty)) # otherwise, only the first subcategory level is enabled

# In our example the relations in our data graph categorizing the pizzas are has_topping, has_kind, has_spiciness, has_base.
map_to_ir_onto_g.add((pizza_ns.has_spiciness, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))
map_to_ir_onto_g.add((pizza_ns.has_kind, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))
map_to_ir_onto_g.add((pizza_ns.has_spiciness, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))
map_to_ir_onto_g.add((pizza_ns.has_base, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))

<Graph identifier=N43e96756c6a74589bae2bd1bc6421b3a (<class 'rdflib.graph.Graph'>)>

In [15]:
# map_to_ir_onto_g.serialize(destination="./data/pizzas_bisou_map_to_ir_onto.ttl", format="turtle")

In [16]:
# Let us add some user searches
searches_graph = Graph()
searches_graph.bind(prefix="pizza", namespace=pizza_ns)
searches_graph.bind(prefix="ir-onto", namespace=ir_onto_ns)

searches_graph.add((pizza_ns._meatyToppingSearch, RDF.type, ir_onto_ns.Search))
searches_graph.add((pizza_ns._meatyToppingSearch, ir_onto_ns.hasSearchCategory, pizza_ns._meat))

searches_graph.add((pizza_ns._onionMushroomToppingSearch, RDF.type, ir_onto_ns.Search))
searches_graph.add((pizza_ns._onionMushroomToppingSearch, ir_onto_ns.hasSearchCategory, pizza_ns._onion))
searches_graph.add((pizza_ns._onionMushroomToppingSearch, ir_onto_ns.hasSearchCategory, pizza_ns._mushroom))

searches_graph.add((pizza_ns._hamToppingSearch, RDF.type, ir_onto_ns.Search))
searches_graph.add((pizza_ns._hamToppingSearch, ir_onto_ns.hasSearchCategory, pizza_ns._ham))

searches_graph.add((pizza_ns._veggieSearch, RDF.type, ir_onto_ns.Search))
searches_graph.add((pizza_ns._veggieSearch, ir_onto_ns.hasSearchCategory, pizza_ns._vegetarianPizza))

searches_graph.add((pizza_ns._veggieWithHamSearch, RDF.type, ir_onto_ns.Search))
searches_graph.add((pizza_ns._veggieWithHamSearch, ir_onto_ns.hasSearchCategory, pizza_ns._vegetarianPizza))
searches_graph.add((pizza_ns._veggieWithHamSearch, ir_onto_ns.hasSearchCategory, pizza_ns._ham))

searches_graph.add((pizza_ns._veggieMeatySearch, RDF.type, ir_onto_ns.Search))
searches_graph.add((pizza_ns._veggieMeatySearch, ir_onto_ns.hasSearchCategory, pizza_ns._vegetarianPizza))
searches_graph.add((pizza_ns._veggieMeatySearch, ir_onto_ns.hasSearchCategory, pizza_ns._meatyPizza))

<Graph identifier=N7628be6914cd4211bf31cd4649be6140 (<class 'rdflib.graph.Graph'>)>

In [17]:
full_graph = data_graph + searches_graph + map_to_ir_onto_g + ir_onto_g 

In [18]:
full_graph.serialize(destination="./data/pizza_bisou_full_graph.ttl", format="turtle")

<Graph identifier=N516a33e6d64f47e2a94219acee328669 (<class 'rdflib.graph.Graph'>)>

In [19]:
# reasoner = "hermit" # "ELK"

# error_output = ""

# input_file = "data/temp/pizza_graph_to_infer.ttl"
# temp_file = "./data/temp/inference_temp.ttl"

# robot_command = [
#     JAVA_EXE,
#     "-jar",
#     ROBOT_JAR,
#     "reason",
#     "--reasoner",
#     reasoner,
#     "--create-new-ontology",
#     "true",
#     # "--annotate-inferred-axioms",
#     # "true",
#     "--input",
#     input_file,
#     "--output",
#     temp_file
# ]

# try:
#     output = subprocess.check_output(
#         robot_command, stderr=subprocess.STDOUT)
# except subprocess.CalledProcessError as e:
#     error_output = e.output.decode()

## Information Retrieval Ontology usage examples

Let us now see some examples of what can we do with the IR ontology.

For these demos we will use a triple store database in a server. 
Another tutorial will explore the same examples with in memory only requirements. 
These demos require OWL inference at query time. We chose the OntoText GraphDB triple store (v10.4.0): <https://graphdb.ontotext.com/documentation/10.0/index.html>. Hence, some pieces of code might be specific to the GraphDB APIs. We will try to flag those as much as possible. Once you the docker container is running you can see their Web API documentation there: <http://localhost:7200/webapi>.
However, GraphDB implements the RDF4J REST API specification (<https://rdf4j.org/documentation/reference/rest-api/>). So we will try to make use of it as much as possible.

We use the Docker GraphDB instance without any license (the GraphDB Free version). Hence, you will need to have Docker installed and running on your computer (See Docker installation procedure: <https://docs.docker.com/get-docker/>). Let's start from there.

1. Download the GraphDB image: `docker pull ontotext/graphdb:10.4.0`
   - OntoText related documentation pointers:
     - <https://github.com/Ontotext-AD/graphdb-docker>
     - <https://hub.docker.com/r/ontotext/graphdb/>
2. Run the image: `docker run -p 127.0.0.1:7200:7200 --name graphdb-ir-onto -t ontotext/graphdb:10.4.0`
3. From now on the rest is in the code.

In [20]:
!docker run -d -p 127.0.0.1:7200:7200 --name graphdb-ir-onto -t ontotext/graphdb:10.4.0

4a82efc1c5b14088601ddb4add3c460657c8275ce3db051124baebfaf6250422


In [21]:
# Setting up the repository we will work with

# Load the repository configuration (WARNING: specific to GraphDB)
repo_config_graph = Graph()
repo_config_graph.parse("./data/ir-onto-demo-graphdb-config.ttl")
repo_config_ttl_string = repo_config_graph.serialize(format="turtle")

headers = {
    "Accept": "application/json",
}
data = {
    "config": ("config.ttl", repo_config_ttl_string)
}

# Uses the GraphDB REST API (WARNING: specific to GraphDB)
r = requests.post("http://localhost:7200/rest/repositories", headers=headers, files=data)
print(r.status_code)
print(r.text)

# Check that the repository is created
r = requests.get('http://localhost:7200/repositories', headers={"Content-type": "application/rdf+xml"})
print(r.status_code)
print(r.text)

# Define the server info for future interactions
DB_IP = "localhost"
DB_PORT = "7200"
DB_URL = f"http://{DB_IP}:{DB_PORT}"
REPOSITORY_ID = "ir-onto-demo"

demo_ns_manager = full_graph.namespace_manager

201

200
uri,id,title,readable,writable
http://localhost:7200/repositories/ir-onto-demo,ir-onto-demo,,true,true



In [22]:
# Lets add the pizza taxonomies to our DB
insert_graph(graph=full_graph, db_url=DB_URL, repo_id=REPOSITORY_ID)

OK


In [23]:
def make_search_the_context(user_search_uri: str) -> None:
    query = f"""
        PREFIX ir-onto: <http://www.msesboue.org/o/ir-ontology#>
        INSERT DATA {{
            <{user_search_uri}> a ir-onto:SearchContext .
        }}
    """

    sparql_update(sparql_query=query, db_url=DB_URL, repo_id=REPOSITORY_ID)

def remove_search_as_context(user_search_uri: str) -> None:
    query = f"""
        PREFIX ir-onto: <http://www.msesboue.org/o/ir-ontology#>
        DELETE DATA {{
            <{user_search_uri}> a ir-onto:SearchContext .
        }}
    """

    sparql_update(sparql_query=query, db_url=DB_URL, repo_id=REPOSITORY_ID)

In [24]:
make_search_the_context(user_search_uri=str(pizza_ns._onionMushroomToppingSearch))

show_instances(class_uri=str(ir_onto_ns.SelectedCategory), ns_manager=demo_ns_manager, db_url=DB_URL, repo_id=REPOSITORY_ID, limit=10)
show_instances(class_uri=str(ir_onto_ns.EnabledCategory), ns_manager=demo_ns_manager, db_url=DB_URL, repo_id=REPOSITORY_ID, limit=10)
show_instances(class_uri=str(ir_onto_ns.CandidateDocument), ns_manager=demo_ns_manager, db_url=DB_URL, repo_id=REPOSITORY_ID, limit=10)

remove_search_as_context(user_search_uri=str(pizza_ns._onionMushroomToppingSearch))

# No candidate document because pizzas are linked to red onion and champignon de paris, not mushroom and onion

OK
Instances of http://www.msesboue.org/o/ir-ontology#SelectedCategory
pizza:_mushroom
pizza:_onion

Instances of http://www.msesboue.org/o/ir-ontology#EnabledCategory
pizza:_carpaccioChampignonDeParis
pizza:_redOnion
pizza:_champignonDeParis

Instances of http://www.msesboue.org/o/ir-ontology#CandidateDocument

OK


In [25]:
make_search_the_context(user_search_uri=str(pizza_ns._veggieSearch))

show_instances(class_uri=str(ir_onto_ns.SelectedCategory), ns_manager=demo_ns_manager, db_url=DB_URL, repo_id=REPOSITORY_ID, limit=10)
show_instances(class_uri=str(ir_onto_ns.EnabledCategory), ns_manager=demo_ns_manager, db_url=DB_URL, repo_id=REPOSITORY_ID, limit=10)
show_instances(class_uri=str(ir_onto_ns.CandidateDocument), ns_manager=demo_ns_manager, db_url=DB_URL, repo_id=REPOSITORY_ID, limit=10)

remove_search_as_context(user_search_uri=str(pizza_ns._veggieSearch))

OK
Instances of http://www.msesboue.org/o/ir-ontology#SelectedCategory
pizza:_vegetarianPizza

Instances of http://www.msesboue.org/o/ir-ontology#EnabledCategory

Instances of http://www.msesboue.org/o/ir-ontology#CandidateDocument
pizza:_biancaCastafiore
pizza:_margherita20
pizza:_quatroFromaggi

OK


Let us add a transversal relation

In [11]:
# Ex: if a pizza is categorised by a meat topping the category meaty pizza should be enabled
# mapping_graph.add((pizza_onto_demo_ns.makesIt, RDFS.subPropertyOf, ir_onto_ns.enablesCategory))

NameError: name 'mapping_graph' is not defined